to do:
- Dokumentere bruk av KI

In [1]:
# imports
import json
import os
import requests
import pandas as pd
import pymongo.errors
import plotly.express as px
import uuid
from bs4 import BeautifulSoup
from io import StringIO
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

## MongoDB

### Set up remote database

In [2]:
# read
with open("../.nosync/mongoDB.json", "r") as file:
    credentials = json.load(file)

uri = (
    "mongodb+srv://medvetslos:"
    + json.load(open("../.nosync/mongoDB.json"))["pwd"]
    + "@ind320-project.lunku.mongodb.net/?retryWrites=true&w=majority&appName=IND320-project"
)

mdb_client = MongoClient(uri, server_api=ServerApi("1"))

try:
    mdb_client.admin.command("ping")
    print("Pinged your deployment. Successfully connected to MongoDB.")
except Exception as exceptionMsg:
    print(exceptionMsg)

Pinged your deployment. Successfully connected to MongoDB.


In [3]:
# Creating collections for municipality data and gas prices
database = mdb_client["IND320-project"]
collection_names = ["muncipalities", "gas"]

for name in collection_names:
    # Checking if collection exists. If not, create the collection.
    try:
        database.create_collection(name)
        print(f"Collection '{name}' was created successfully.")
    except pymongo.errors.CollectionInvalid:
        print(f"Collection '{name}' already exists.")

municipalities = database["municipalities"]
gas = database["gas"]


Collection 'muncipalities' was created successfully.
Collection 'gas' already exists.


## Cassandra

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster(["localhost"], port=9042)
session = cluster.connect()
keyspace = "ind320_project"
session.execute(
    "CREATE KEYSPACE IF NOT EXISTS" + 
    " " + 
    keyspace + 
    " " + 
    "WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1};"
)

session.set_keyspace(keyspace)

## Webscraping



In [18]:
webscrape_url = "https://en.wikipedia.org/wiki/List_of_municipalities_of_Denmark"

page = requests.get(webscrape_url)
soup = BeautifulSoup(page.content, "html.parser")
wiki_table = soup.find("table", attrs={"class": "wikitable sortable"})

df_municipalities = pd.read_html(StringIO(str(wiki_table)))[0]
records_municipalities = df_municipalities.to_dict("records")

LAU_1_code = df_municipalities.columns.tolist()[0]

# WRITE TO FUNCTION!
# check if data we are writing already exists
existing_entries = list(
    database["municipalities"].find(
        {
            LAU_1_code: {
                "$in": [record[LAU_1_code] for record in records_municipalities]
            }
        }
    )
)

# if new data,
new_entries = [
    entry for entry in records_municipalities
    if not any(existing_entry[LAU_1_code] == entry[LAU_1_code] for existing_entry in existing_entries)
]

# Writing to MongoDB
if len(new_entries) > 0:
    database["municipalities"].insert_many(new_entries)
    print("Data successfully written into the collection.")
else:
    print("No new data to be inserted into the collection.")

# database["municipalities"].delete_many({}) # delete all records

No new data to be inserted into the collection.


## API

In [5]:
api_url = "https://api.energidataservice.dk/dataset/"
filtering = "?offset=0&start=2022-01-01T00:00&end=2023-01-01T00:00"

api_datasets = {
    "remote": {"gas": "GasDailyBalancingPrice"},
    "local": {
         "production": "ProductionMunicipalityHour", 
         "consumption": "ConsumptionIndustry",
         "prodcons": "ProductionConsumptionSettlement"
    }
}

In [85]:
def get_json_data(dataset: str):
    return requests.get(api_url + dataset + filtering).json()["records"]

### To MongoDB

In [9]:
df_gas = pd.DataFrame.from_records(get_json_data(api_datasets["remote"]["gas"]))

In [17]:
df_gas["GasDay"] = pd.to_datetime(df_gas["GasDay"])
gas_data = df_gas.to_dict("records")
database["gas"].insert_many(gas_data)

InsertManyResult([ObjectId('6704f9f14069935186461552'), ObjectId('6704f9f14069935186461553'), ObjectId('6704f9f14069935186461554'), ObjectId('6704f9f14069935186461555'), ObjectId('6704f9f14069935186461556'), ObjectId('6704f9f14069935186461557'), ObjectId('6704f9f14069935186461558'), ObjectId('6704f9f14069935186461559'), ObjectId('6704f9f1406993518646155a'), ObjectId('6704f9f1406993518646155b'), ObjectId('6704f9f1406993518646155c'), ObjectId('6704f9f1406993518646155d'), ObjectId('6704f9f1406993518646155e'), ObjectId('6704f9f1406993518646155f'), ObjectId('6704f9f14069935186461560'), ObjectId('6704f9f14069935186461561'), ObjectId('6704f9f14069935186461562'), ObjectId('6704f9f14069935186461563'), ObjectId('6704f9f14069935186461564'), ObjectId('6704f9f14069935186461565'), ObjectId('6704f9f14069935186461566'), ObjectId('6704f9f14069935186461567'), ObjectId('6704f9f14069935186461568'), ObjectId('6704f9f14069935186461569'), ObjectId('6704f9f1406993518646156a'), ObjectId('6704f9f140699351864615

### To Cassandra

In [86]:
df_production = pd.DataFrame.from_records(get_json_data(api_datasets["local"]["production"]))
df_consumption = pd.DataFrame.from_records(get_json_data(api_datasets["local"]["consumption"]))
df_prodcons = pd.DataFrame.from_records(get_json_data(api_datasets["local"]["prodcons"]))

In [87]:
df_production.dtypes

HourUTC                     object
HourDK                      object
MunicipalityNo              object
SolarMWh                   float64
OffshoreWindLt100MW_MWh    float64
OffshoreWindGe100MW_MWh    float64
OnshoreWindMWh             float64
ThermalPowerMWh            float64
dtype: object

In [88]:
df_production['HourDK'] = pd.to_datetime(df_production['HourDK'])
df_production['HourUTC'] = pd.to_datetime(df_production['HourUTC'])
df_production['MunicipalityNo'] = df_production['MunicipalityNo'].astype(int)

In [94]:
df_consumption.dtypes

hourutc           datetime64[ns]
hourdk            datetime64[ns]
municipalityno             int64
branche                   object
consumptionkwh           float64
dtype: object

In [90]:
df_consumption['HourDK'] = pd.to_datetime(df_consumption['HourDK'])
df_consumption['HourUTC'] = pd.to_datetime(df_consumption['HourUTC'])
df_consumption['MunicipalityNo'] = df_consumption['MunicipalityNo'].astype(int)
df_consumption['Branche'] = df_consumption['Branche'].astype(str)

In [91]:
df_prodcons.dtypes

HourUTC                        object
HourDK                         object
PriceArea                      object
CentralPowerMWh               float64
LocalPowerMWh                 float64
CommercialPowerMWh            float64
LocalPowerSelfConMWh          float64
OffshoreWindLt100MW_MWh       float64
OffshoreWindGe100MW_MWh       float64
OnshoreWindLt50kW_MWh         float64
OnshoreWindGe50kW_MWh         float64
HydroPowerMWh                 float64
SolarPowerLt10kW_MWh          float64
SolarPowerGe10Lt40kW_MWh      float64
SolarPowerGe40kW_MWh          float64
SolarPowerSelfConMWh          float64
UnknownProdMWh                float64
ExchangeNO_MWh                float64
ExchangeSE_MWh                float64
ExchangeGE_MWh                float64
ExchangeNL_MWh                float64
ExchangeGB_MWh                 object
ExchangeGreatBelt_MWh         float64
GrossConsumptionMWh           float64
GridLossTransmissionMWh       float64
GridLossInterconnectorsMWh    float64
GridLossDist

In [92]:
df_prodcons['HourDK'] = pd.to_datetime(df_prodcons['HourDK'])
df_prodcons['HourUTC'] = pd.to_datetime(df_prodcons['HourUTC'])
df_prodcons['PriceArea'] = df_prodcons['PriceArea'].astype(str)
df_prodcons['ExchangeGB_MWh'] = df_prodcons["ExchangeGB_MWh"].astype(float)

In [93]:
# Had to set column names to lowercase to be able to write through PySpark
df_prodcons = df_prodcons.rename(columns=str.lower)
df_production = df_production.rename(columns=str.lower)
df_consumption = df_consumption.rename(columns=str.lower)

In [95]:
# Prompt: Can you write me a function which makes the table creation query from a 
#         Pandas DataFrame which assigns the correct datatype to the Cassandra table.
#         The primary key should be named id and be of type timeuuid
def create_cassandra_table_query(df, keyspace, table_name):
    # Define mapping between Pandas and Cassandra datatypes
    dtype_mapping = {
        'int64': 'int',
        'float64': 'double',
        'object': 'text',
        'bool': 'boolean',
        'datetime64[ns]': 'timestamp'
    }
    
    # Start constructing the CREATE TABLE query
    query = f"CREATE TABLE IF NOT EXISTS {keyspace}.{table_name} (\n"
    
    # Add primary key column with timeuuid
    columns = ["id timeuuid"]
    
    # Add remaining columns with mapped Cassandra data types
    for col, dtype in df.dtypes.items():
        if col != "id":  # Exclude 'id' to avoid duplication
            cassandra_type = dtype_mapping.get(str(dtype), 'text')  # Default to 'text' if type is unrecognized
            columns.append(f"{col} {cassandra_type}")
    
    # Join columns with commas and specify primary key as 'id'
    columns_str = ",\n    ".join(columns)
    query += f"    {columns_str},\n"
    query += f"    PRIMARY KEY (id)\n);"
    
    return query


In [96]:
production_table = create_cassandra_table_query(df_production, keyspace, "production")
consumption_table = create_cassandra_table_query(df_consumption, keyspace, "consumption") 
prodcons_table = create_cassandra_table_query(df_prodcons, keyspace, "prodcons") 

for table_query in [production_table, consumption_table, prodcons_table]:
    session.execute(table_query)

In [97]:
query = f"SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'ind320_project'"
keyspace = "ind320_project"
# Execute the query
rows = session.execute(query)

# Print the table names
print(f"Tables in keyspace '{keyspace}':")
for row in rows:
    print(row.table_name)
    # session.execute(f"DROP TABLE IF EXISTS {keyspace}.{row.table_name}")

Tables in keyspace 'ind320_project':
consumption
prodcons
production


## Spark to Cassandra

In [6]:
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/"
os.environ["PYSPARK_PYTHON"] = "python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "python" 
os.environ["PYSPARK_HADOOP_VERSION"] = "without"

In [7]:
spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.1').\
    config('spark.cassandra.connection.host', 'localhost').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').\
    config("spark.driver.memory", "4g").\
    config("spark.executor.memory", "4g").\
    config("spark.task.maxFailures", "10").\
    config("spark.sql.shuffle.partitions", "200").\
    getOrCreate()


24/10/08 13:45:44 WARN Utils: Your hostname, Aarons-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.11.132 instead (on interface en0)
24/10/08 13:45:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/aaron/.ivy2/cache
The jars for the packages stored in: /Users/aaron/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a8a7378-74b7-4a46-9e69-55c7dbfc17b5;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central


:: loading settings :: url = jar:file:/Users/aaron/Documents/IND320_projects/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.github.spotbugs#spotbugs-annotations;3.1.12 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.datastax.oss#java-driver-mapper-runtime;4.13.0 in central
	found com.datastax.oss#java-driver-query-builder;4.13.0 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.sca

In [100]:
def generate_timeuuid():
    return str(uuid.uuid1())

timeuuid_udf = udf(generate_timeuuid, StringType())

spark.createDataFrame(df_production).withColumn("id", timeuuid_udf())\
    .write.format("org.apache.spark.sql.cassandra")\
    .mode("append")\
    .options(table="production", keyspace="ind320_project")\
    .save()

24/10/08 13:15:07 WARN TaskSetManager: Stage 3 contains a task of very large size (5307 KiB). The maximum recommended task size is 1000 KiB.


In [101]:
spark.createDataFrame(df_consumption).withColumn("id", timeuuid_udf())\
    .write.format("org.apache.spark.sql.cassandra")\
    .mode("append")\
    .options(table="consumption", keyspace="ind320_project")\
    .save()

24/10/08 13:16:11 WARN TaskSetManager: Stage 4 contains a task of very large size (9911 KiB). The maximum recommended task size is 1000 KiB.


In [102]:
spark.createDataFrame(df_prodcons).withColumn("id", timeuuid_udf())\
    .write.format("org.apache.spark.sql.cassandra")\
    .mode("append")\
    .options(table="prodcons", keyspace="ind320_project")\
    .save()

Idea now:
- Use Spark/Cassandra to retrieve data rather than through the API

# Plotting

## Energy exchange to Norway

In [8]:
def create_view_from_cas(view_name:str, table: str, keyspace: str):
    spark.read.format("org.apache.spark.sql.cassandra")\
        .options(table=table, keyspace=keyspace)\
        .load()\
        .createOrReplaceTempView(view_name)
    print(f"View '{view_name}' created.")

In [9]:
spark.read.format("org.apache.spark.sql.cassandra")\
    .options(table="prodcons", keyspace=keyspace)\
    .load()\
    .createOrReplaceTempView("prodcons_view")

24/10/08 13:45:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [105]:
sdf_prodcons = spark.sql("SELECT * FROM prodcons_view").toPandas()

In [106]:
sdf_prodcons = sdf_prodcons.sort_values(by="hourdk", ascending=True)
df_exchangeno = sdf_prodcons[["hourdk", "exchangeno_mwh"]].copy()

In [107]:
df_exchangeno_agg = df_exchangeno.groupby("hourdk").aggregate({"exchangeno_mwh": "sum"})

In [108]:
exchangeNo_fig = px.line(df_exchangeno_agg, title = "Exchange of Energy to Norway (MWh)")
exchangeNo_fig.show()


## Gas prices

In [109]:
gas = database["gas"]
gas_records = [document for document in gas.find()]
df_gas = pd.DataFrame.from_records(gas_records)
df_gas = df_gas.sort_values("GasDay", ascending=True)
df_gas.head()

,_id,GasDay,EEXSpotIndexEUR_MWh,EEXWithinDayEUR_MWh,ExchangeRateEUR_DKK,SalesPriceDKK_kWh,PurchasePriceDKK_kWh,NeutralGasPriceDKK_kWh,EEXHighestPricePurchaseDKK_kWh,EEXLowestPriceSaleDKK_kWh,MarginalSalePriceDKK_kWh,MarginalPurchasePriceDKK_kWh,THEPriceDKK_kWh
91,6704f9f140699351864615ad,2022-10-01,168.168,165.000,743.65,1.2332,1.2209,1.2270,NaN,NaN,1.2332,1.2209,1.2239
90,6704f9f140699351864615ac,2022-10-02,168.168,157.742,743.65,1.1789,1.1672,1.1730,NaN,NaN,1.1789,1.1672,1.2239
89,6704f9f140699351864615ab,2022-10-03,167.210,141.581,743.66,1.0581,1.0476,1.0529,NaN,NaN,1.0581,1.0476,1.2203
88,6704f9f140699351864615aa,2022-10-04,147.942,117.112,743.74,0.8754,0.8667,0.8710,NaN,0.3719,0.8754,0.3719,1.0478
87,6704f9f140699351864615a9,2022-10-05,119.772,82.244,743.88,0.6149,0.6087,0.6118,NaN,NaN,0.6149,0.6087,0.7978


In [110]:
price_fig = px.line(df_gas, x="GasDay", y=["SalesPriceDKK_kWh", "PurchasePriceDKK_kWh"], 
                    title="Sale and pruchase prices of Gas (DKK)")
price_fig.show()

In [10]:
create_view_from_cas("production_view", "production", keyspace)
sdf_production = spark.sql("SELECT * FROM production_view").toPandas()

View 'production_view' created.


In [31]:
sdf_production[[
    # 'id', 
    'hourdk', 
    # 'hourutc', 
    'municipalityno', 'offshorewindge100mw_mwh',
    'offshorewindlt100mw_mwh', 'onshorewindmwh', 'solarmwh',
    'thermalpowermwh'
]].head()

,hourdk,municipalityno,offshorewindge100mw_mwh,offshorewindlt100mw_mwh,onshorewindmwh,solarmwh,thermalpowermwh
0,2022-03-11 12:00:00,760,NaN,NaN,216.128232,34.555,8.936840
1,2022-06-06 14:00:00,183,NaN,NaN,0.000000,0.286,NaN
2,2022-12-12 21:00:00,480,NaN,NaN,0.312910,0.000,2.023160
3,2022-10-11 00:00:00,167,NaN,5.3713,0.193480,0.000,0.000000
4,2022-07-02 08:00:00,851,NaN,NaN,26.782163,5.871,356.511291


In [12]:
create_view_from_cas("consumption_view", "consumption", keyspace)
sdf_consumption = spark.sql("SELECT * FROM consumption_view").toPandas()

View 'consumption_view' created.


In [30]:
sdf_consumption[[
    # 'id', 
    'branche', 
    'consumptionkwh', 
    'hourdk', 
    # 'hourutc',
    'municipalityno'
]].head()

,branche,consumptionkwh,hourdk,municipalityno
0,Offentligt,14398.974,2022-01-14 17:00:00,851
1,Erhverv,34227.385,2022-03-14 02:00:00,779
2,Privat,10568.550,2022-07-09 14:00:00,217
3,Erhverv,20806.468,2022-06-15 14:00:00,430
4,Offentligt,13097.933,2022-06-20 16:00:00,851


In [15]:
create_view_from_cas("prodcons_view", "prodcons", keyspace)
sdf_prodcons = spark.sql("SELECT * FROM prodcons_view").toPandas()

View 'prodcons_view' created.


In [32]:
sdf_prodcons[[
    # 'id', 
    'centralpowermwh', 'commercialpowermwh', 
    # 'exchangegb_mwh','exchangege_mwh', 'exchangegreatbelt_mwh', 
    # 'exchangenl_mwh', 'exchangeno_mwh', 'exchangese_mwh', 
    # 'gridlossdistributionmwh', 'gridlossinterconnectorsmwh', 'gridlosstransmissionmwh',
    # 'grossconsumptionmwh', 
    'hourdk', 
    # 'hourutc', 
    'hydropowermwh',
    'localpowermwh', 
    'localpowerselfconmwh', 'offshorewindge100mw_mwh',
    'offshorewindlt100mw_mwh', 'onshorewindge50kw_mwh',
    'onshorewindlt50kw_mwh', 
    # 'powertoheatmwh', 
    'pricearea',
    'solarpowerge10lt40kw_mwh', 'solarpowerge40kw_mwh',
    'solarpowerlt10kw_mwh', 'solarpowerselfconmwh', 'unknownprodmwh'
]].head()

,centralpowermwh,commercialpowermwh,hourdk,hydropowermwh,localpowermwh,localpowerselfconmwh,offshorewindge100mw_mwh,offshorewindlt100mw_mwh,onshorewindge50kw_mwh,onshorewindlt50kw_mwh,pricearea,solarpowerge10lt40kw_mwh,solarpowerge40kw_mwh,solarpowerlt10kw_mwh,solarpowerselfconmwh,unknownprodmwh
0,3.321915,234.585354,2022-05-25 19:00:00,0.0,29.123822,4.638933,322.691581,16.4199,199.938341,0.482856,DK2,0.235912,14.675579,2.720010,5.28945,0.02541
1,235.182031,82.231489,2022-12-29 05:00:00,0.0,41.990846,17.678838,782.917060,27.1770,558.002769,2.080167,DK2,0.008782,0.000010,0.061561,0.00000,0.01559
2,380.359606,237.140916,2022-11-28 16:00:00,0.0,64.648456,20.229570,865.486869,37.0681,486.056078,1.986844,DK2,0.026406,0.000060,0.099967,0.00000,0.00901
3,31.300468,245.188237,2022-10-16 19:00:00,0.0,48.419679,6.725389,598.270387,21.5596,395.901284,0.995266,DK2,0.008316,0.000414,0.158370,0.00000,0.00288
4,465.467082,147.803214,2022-02-18 21:00:00,0.0,65.017969,7.386351,631.759501,23.1076,296.390523,2.071780,DK2,0.009336,0.000000,0.024509,0.00000,0.02847


"select random values for each pair"

oversatt betyr det å velge en tilfeldig:
- municpalityno og productiontype kolonne
- municipalityno og branche
- pricearea og productiontype
